In [1]:
""""
Copyright (c) Meta Platforms, Inc. and affiliates.
All rights reserved.

This source code is licensed under the license found in the
LICENSE file in the root directory of this source tree.
"""

import glob
import json
import numpy as np
import os
import pandas as pd
import re
import torch

# SALSA Attack Analysis

In [2]:
paths = ["/path/to/logs/he_29/*/", "/path/to/logs/he_50_ternary/*/"]
file_paths = [x for path in paths for x in glob.glob(path)]
print(len(file_paths))
information = {}

180


In [3]:
for file_path in file_paths:
  exp_id = os.path.basename(file_path)
  log_file_path = os.path.join(file_path, "train.log")
  ckpt_file_path = os.path.join(file_path, "checkpoint.pth")
  information[file_path] = {}
  with open(log_file_path, 'r') as file1:
    lines = file1.readlines()
    for line in lines:
      if not line:
          break
      if "Predicted" in line:
        information[file_path]["success"] = True
        for key, val in torch.load(ckpt_file_path,map_location=torch.device('cpu'))["params"].items():
          information[file_path][key] = val
      if "[hours:" in line:
        information[file_path]["time"] = float(line.split(" ")[-3].strip(","))
        information[file_path]["epoch"] = int(line.split(" ")[-1].strip("]\n"))
      if "Error" in line:
        information[file_path]["error"] = line
        print(line)

In [4]:
df = pd.DataFrame(information).T
try:
  df.success.fillna(False, inplace=True)
except AttributeError:
  print("no successes for following experiments: ", paths)

# remove the ones that don't have "Attack took", they failed because of something else
df = df[df.time.notna()]
print(f"total n successful: {len(df[df.success])}")
print(f"hamming weights recovered: {sorted(df[df.success].hamming.unique())}")
print()


for exp in df.exp_name.unique():
    success_df = df[(df.success) & (df.exp_name == exp)]
    for h in sorted(success_df.hamming.unique()):
        new_df = success_df[success_df.hamming == h]
        print(exp, "h: ", h, "recoveries: ", len(new_df))
        print("min recovery time: ", new_df["time"].min())
        print("min num epochs: ", new_df["epoch"].min())
        print()

total n successful: 21
hamming weights recovered: [7, 8, 9, 10, 11, 12, 13, 14, 16, 17]

he_29 h:  7 recoveries:  3
min recovery time:  2.8
min num epochs:  2

he_29 h:  8 recoveries:  1
min recovery time:  15.9
min num epochs:  12

he_29 h:  9 recoveries:  2
min recovery time:  3.1
min num epochs:  3

he_29 h:  10 recoveries:  1
min recovery time:  17.8
min num epochs:  11

he_50_ternary h:  11 recoveries:  3
min recovery time:  4.7
min num epochs:  3

he_50_ternary h:  12 recoveries:  3
min recovery time:  1.1
min num epochs:  1

he_50_ternary h:  13 recoveries:  4
min recovery time:  6.3
min num epochs:  3

he_50_ternary h:  14 recoveries:  1
min recovery time:  3.9
min num epochs:  2

he_50_ternary h:  16 recoveries:  2
min recovery time:  20.4
min num epochs:  9

he_50_ternary h:  17 recoveries:  1
min recovery time:  5.3
min num epochs:  4



# CC Attack Analysis 

In [20]:
expnames = ["cc_26_ternary_he_final", "cc_29_ternary_he_final", "cc_50_ternary_he_final"]
paths = [f"/path/to/logs/{expname}/**/*.stdout" for expname in expnames]
file_paths = [x for path in paths for x in glob.glob(path)]
print(len(file_paths))
information = {}

280


In [21]:
for file_path in file_paths:
  information[file_path] = {}

  with open(file_path, "r") as f:
    lines = f.readlines()
    for line in lines:
      if "SUCCESS" in line:
        information[file_path]["success"] = True
      if "grid_params" in line:
        for key, val in json.loads(line[line.index("{"):]).items():
          information[file_path][key] = val
      if "swept_params" in line:
        for key, val in json.loads(line[line.index("{"):]).items():
          information[file_path][key] = val
      if "Attack took" in line:
        information[file_path]["time"] = float(line.split(" ")[-2]) / 3600.0
      match = re.search(r'"full_hw": (\d+)', line)
 
      if match:
        full_hw = int(match.group(1)) 
        information[file_path]["bf_hw"] = full_hw
      if "minhi = " in line:
        _, minhi = line.split("=")
        minhi = int(minhi.split(",")[0].strip())
        information[file_path]["minhi"] = minhi

df = pd.DataFrame(information).T
try:
  df.success.fillna(False, inplace=True)
except AttributeError:
  print("no successes for following experiments: ", expnames, len(file_paths))

In [26]:
for exp in df.exp_name.unique():
  mini_df = df[df.exp_name == exp]
  success_df = mini_df[mini_df.success]
  print(exp)
  print(f"n successful: {len(success_df)}")
  print(f"hamming weight {sorted(success_df.full_hw.unique())}")
  print(mini_df.groupby("bf_hw").sum(numeric_only=True))
  for hw in sorted(success_df.bf_hw.unique()):
    mini_success_df = success_df[success_df.bf_hw == hw]
    # print(mini_success_df[["time", "bf_hw", "seed"]].sort_values(["bf_hw", "time"]))
    print("hamming weight: ", hw)
    print("min recovery time: ", mini_success_df.time.min())
  print()

cc_26_ternary_he_final
n successful: 24
hamming weight [4, 5, 6, 7]
       success
bf_hw         
4           10
5            7
6            5
7            2
8            0
9            0
hamming weight:  4
min recovery time:  0.028533690306875442
hamming weight:  5
min recovery time:  0.04477666106488969
hamming weight:  6
min recovery time:  0.39132829633024
hamming weight:  7
min recovery time:  0.4141557166311476

cc_29_ternary_he_final
n successful: 32
hamming weight [7, 8, 9, 12]
       success
bf_hw         
7           10
8           11
9           10
10           0
11           0
12           1
13           0
hamming weight:  7
min recovery time:  0.03257233096493615
hamming weight:  8
min recovery time:  0.07510364804002974
hamming weight:  9
min recovery time:  0.03067563282118903
hamming weight:  12
min recovery time:  0.13082878172397613

cc_50_ternary_he_final
n successful: 70
hamming weight [11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
       success
bf_hw         
11        